In [1]:
from Project_code import *

board = chess.Board()

pre_mate = chess.Board('rnbqkbnr/pppp1ppp/8/4p3/6P1/5P2/PPPPP2P/RNBQKBNR b KQkq - 0 2')


In [2]:
models = loadModels()

NNnames=["Single256","Single128","Single64","Single32","Pair256","Double128","Single256_Double256_128","Single128_Double128"]
NNminply = ["0","15","30","45","60"]

engines = [(i,j) for i in NNnames for j in NNminply]

In [3]:
nsearcher = SearcherDirector.minimax_NN(3)
feature1 = FeatureDirector.PieceValue(0)
feature2 = FeatureDirector.PawnAdvancement(0.2)
features = [feature1,feature2]
evaluator = EvaluatorDirector.sqrt_dif(features)
msearcher = SearcherDirector.AB_pruning(3)
MBot = BotPlayer("Manual",evaluator,msearcher)

for i in range(40):
    mod1 = engines[i]
    print(mod1,"Manual")
    NBot1 = BotPlayer("NN",models[f"{mod1[0]}_{mod1[1]}"],nsearcher)
    
    simulateMultipleGames(NBot1,MBot,2)


for i in range(40):
    mod1 = engines[i]
    for j in range(i+1,40):
        mod2 = engines[j]
        print(mod1,mod2)
        NBot1 = BotPlayer("NN",models[f"{mod1[0]}_{mod1[1]}"],nsearcher)
        NBot2 = BotPlayer("NN",models[f"{mod2[0]}_{mod2[1]}"],nsearcher)
        print(simulateMultipleGames(NBot1,NBot2,2))

('Single256', '0') Manual
[0, 2, 0]
('Single256', '15') Manual
[0, 2, 0]
('Single256', '30') Manual
[0, 2, 0]
('Single256', '45') Manual
[0, 2, 0]
('Single256', '60') Manual
[0, 2, 0]
('Single128', '0') Manual
[0, 2, 0]
('Single128', '15') Manual
[0, 2, 0]
('Single128', '30') Manual
[0, 2, 0]
('Single128', '45') Manual
[0, 2, 0]
('Single128', '60') Manual
[0, 2, 0]
('Single64', '0') Manual
[0, 2, 0]
('Single64', '15') Manual
[0, 2, 0]
('Single64', '30') Manual
[0, 2, 0]
('Single64', '45') Manual


KeyboardInterrupt: 

In [6]:
simulateChessGame(NNBot,NNBot)

(0, Board('1n2k1nr/6p1/r3b1Pp/p1b4P/P7/3p4/1B1N4/1Qn1KBNR w Kk - 16 32'))

In [13]:
searcher = SearcherDirector.minimax(3)
MBot = BotPlayer("Manual",evaluator,searcher)
t = time.time()
MBot.doMove(board)
print(time.time()-t)

searcher = SearcherDirector.AB_prunning(3)
MBot = BotPlayer("Manual",evaluator,searcher)
t = time.time()
MBot.doMove(board)
print(time.time()-t)

3.300286293029785
4.40035343170166


In [10]:
simulateMultipleGames(NNBot,MBot,1)

[0, 0, 1]


In [5]:
simulateChessGame(MBot,NNBot)

(1, Board('1R1Q1br1/8/8/3Qk3/4P3/2P3P1/3P2pP/1NB1KBNR b K - 4 19'))

In [6]:
simulateChessGame(NNBot,MBot)

(-1, Board('3qk3/6p1/n7/1p2b3/1p2Pn2/5P2/K5Br/r2q4 w - - 2 42'))

In [7]:
simulateChessGame(MBot,NNBot)

(1, Board('r2Q4/1k1Q4/8/pP2pp1p/2P5/1P6/3P1PPP/RN2KBNR b KQ - 0 18'))

In [8]:
simulateChessGame(NNBot,MBot)

(-1, Board('1n2k1n1/1pp3p1/r2b4/8/1ppK2p1/3q4/8/r7 w - - 2 24'))

In [4]:
MBot.doMove(chess.Board("rn2k1n1/1pp3p1/8/P2P4/1bp5/8/4K3/r5q1 b q - 1 27"))

UnboundLocalError: local variable 'white' referenced before assignment